In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import quandl

from data_apps_aws.src.sql import get_db_engine, get_db_data
from data_apps_aws.src.passwords import get_api_token

In [ ]:
plt.rcParams['figure.figsize'] = 14, 6
sns.set()

In [ ]:
quandl_api_token = get_api_token('quandl')

In [ ]:
raw_data = quandl.get("FED/PARAMS", authtoken=quandl_api_token)

In [ ]:
raw_data

In [ ]:
maturs = [1, 2, 3, 5, 7, 10, 20, 30]
matur_labels = [str(this_matur) + ' YR' for this_matur in maturs]

In [ ]:
raw_data = data_fetching_tools.get_multiple_joint_datasets(['BETA0', 'BETA1', 'BETA2', 'BETA3', 'TAU1', 'TAU2'], 'ecb')

In [ ]:
from pandasdmx import Request

In [ ]:
def get_ecb_long_term_yields(ts_key, resource_id ='IRS', params=None):
    """
    Resource ID:
    - IRS: Interest rate statistics
    - 
    """
    ecb = Request('ECB')
    
    if params is None:
        data_response = ecb.data(resource_id = resource_id,
                                 key={'REF_AREA': ts_key,
                                     'IR_TYPE': 'L'})
    else:
        data_response = ecb.data(resource_id = resource_id,
                                 key={'REF_AREA': ts_key,
                                     'IR_TYPE': 'L'},
                                 params=params
                                )
        
    raw_data = data_response.data
    series_tuple = (s for s in raw_data.series)
    data = data_response.write(series_tuple)
    
    return data

In [ ]:
data_response = ecb.data(resource_id = 'IRS',
                                 key={'REF_AREA': ['DE'],
                                     'IR_TYPE': 'L'})

In [ ]:
raw_data = data_response.data

In [ ]:
raw_data

In [ ]:
'G_N_C'

In [ ]:
list(raw_data.series)[0].key

In [ ]:
long_term_yields_no_euro = get_ecb_long_term_yields(list_of_areas_no_euro, params={'startPeriod': '2016'})

In [ ]:
ecb = Request('ECB')
exr_flow = ecb.dataflow('EXR')

In [ ]:
ecb = Request('ECB')
exr_flow = ecb.dataflow()

## Get metadata

In [ ]:
data_df = long_term_yields_no_euro.copy()

In [ ]:
ecb = Request('ECB')

In [ ]:
this_resource_id = 'IRS'

def get_resource_name(this_resource_id):
    
    exr_flow = ecb.dataflow()
    resource_types = exr_flow.write().dataflow
    this_resource_name = exr_flow.write().dataflow.loc[this_resource_id, 'name']
    
    return this_resource_name

In [ ]:
get_resource_name(this_resource_id)

In [ ]:
def get_concept_list_from_data(data_df):
    
    return list(data_df.columns.names)

In [ ]:
list_of_data_concepts = get_concept_list_from_data(data_df)
list_of_data_concepts

In [ ]:
def get_resource_metadata_decodes(this_resource_id):
    
    this_resource_decodes = ecb.dataflow(this_resource_id).write().codelist
    
    return this_resource_decodes

In [ ]:
decodes_df = get_resource_metadata_decodes(this_resource_id)
decodes_df.head(2)

In [ ]:
decodes_df.shape

In [ ]:
def get_concept_decodes(decodes_df, concepts):
    
    concepts_labels = decodes_df.loc[(concepts, concepts), ['name']]
    concepts_labels.index = concepts_labels.index.get_level_values(0)
    
    return concepts_labels

In [ ]:
concept_decode_mapping = get_concept_decodes(decodes_df, list_of_data_concepts)
concept_decode_mapping

In [ ]:
all_maps = None

for this_concept in list_of_data_concepts:
    
    these_occurring_vals = data_df.columns.get_level_values(this_concept).unique().values
    
    # get occurring values
    this_map = decodes_df.loc[this_concept, :].loc[these_occurring_vals, 'name'].to_frame().reset_index()
    this_map.rename({'index': 'code'}, axis=1, inplace=True)
    this_map['concept'] = this_concept
    
    if all_maps is None:
        all_maps = this_map
    else:
        all_maps = pd.concat([all_maps, this_map], axis=0)


df_merge = concept_decode_mapping.reset_index()
df_merge.columns = ['concept', 'concept_name']

all_maps.merge(df_merge).loc[:, ['concept', 'concept_name', 'code', 'name']]

In [ ]:
def decode_concept_values(this_concept, these_values):
    
    if type(these_values) == 'str':
        map_ss = all_maps.query('concept == @this_concept & code == @these_values')
    else:
        map_ss = all_maps.query('concept == @this_concept & code in @these_values')
        
    return map_ss.set_index('code').loc[these_values, 'name']

In [ ]:
this_concept = 'REF_AREA'
this_concept_values = data_df.columns.get_level_values(this_concept).values

decode_concept_values('REF_AREA', this_concept_values)

In [ ]:
this_concept = 'MATURITY_CAT'
this_concept_values = data_df.columns.get_level_values(this_concept).values

decode_concept_values(this_concept, this_concept_values)

In [ ]:
data_df.head(3)

## Download data and translate headers

In [ ]:
long_term_yields_no_euro.head(3)

In [ ]:

list_of_areas_no_euro = ['BG', 'CZ', 'DK', 'GB', 'HR', 'HU', 'PL', 'RO', 'SE']

long_term_yields_no_euro = get_ecb_long_term_yields(list_of_areas_no_euro)
long_term_yields_no_euro.columns = long_term_yields_no_euro.columns.levels[1].values

In [ ]:
long_term_yields_no_euro.plot()

In [ ]:
list_of_areas_w_euro = ['AT', 'BE', 'CY', 'DE', 'ES', 'FI', 'FR', 'GR', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PT', 'SI', 'SK']

long_term_yields_w_euro = get_ecb_long_term_yields(list_of_areas_w_euro)
long_term_yields_w_euro.columns = long_term_yields_w_euro.columns.levels[1].values

In [ ]:
long_term_yields_no_euro

In [ ]:
long_term_yields_w_euro.plot()

In [ ]:

raw_ecb_triple_a_data = data_fetching_tools.get_multiple_joint_datasets(['BETA0', 'BETA1', 'BETA2', 'BETA3', 'TAU1', 'TAU2'], 'ecb')
raw_ecb_triple_a_data.head(4)

In [ ]:

raw_ecb_all_ratings_data = data_fetching_tools.get_multiple_joint_datasets(['BETA0', 'BETA1', 'BETA2', 'BETA3', 'TAU1', 'TAU2'], 'ecb', instrument='G_N_C')
raw_ecb_all_ratings_data.head(4)